# Thesis Macroeconomic effect of Trump
## Table of Contents
* [Explanation](#chapter0)
* [1. Data Generating Process](#chapter1)
    * [1.1. Import Packages](#section1_1)
    * [1.2. Set Initial Parameters](#section1_2)
    * [1.3. Define DGP Functions](#section1_3)
    * [1.4. Example Data plot](#section1_4)
* [2. Synthetic Control Class definitions and functions](#chapter2)
    * [2.1 Synthetic Control Class definition](#section2_1)
    * [2.1 Help functions simulation](#section2_2)
* [3. Simulation](#chapter3)
    * [3.1. Different Variables](#section3_1)
    * [3.2. Different V Matrices](#section3_2)
    * [3.3. Different Weight Constraints](#section3_3)

## Explanation <a class="anchor" id="chapter0"></a>

## 1. Data Generating Process <a class="anchor" id="chapter1"></a>
### 1.1. Import Packages <a class="anchor" id="section1_1"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import minimize
from cvxopt import solvers, matrix
solvers.options['show_progress'] = False

### 1.2. Set Initial Parameters <a class="anchor" id="section1_2"></a>

In [ ]:
# DGP parameters
J = 20 # No. of countries, J-1 no. of donor countries
T = 25 # Total periods
T0 = 15 # Pre-treatment period
K = 10

# Simulation parameters
TE_sizes = [0, 0.1, 0.25, 0.5, 1] # Real treatment effects
n_sims = 1000

# SCM parameters
maxiter = 1000
maxfev = 1000
t0 = 10 # Cross-validation parameter

### 1.3. Define DGP function <a class="anchor" id="section1_3"></a>

In [ ]:
def dgp(J, T0, T, process, treatmenteffect=1000):
    
    # Parameter to generate our AR processes
    rho = 0.5
    sd_rho = np.sqrt(1-rho**2)
    
    # Create unit identifiers (names)
    unit_ids = [chr(j + 64) for j in range(1, J+1)]

    # Create an empty dataframe
    data = pd.DataFrame(np.nan, index=np.arange(T), columns=unit_ids)

    # Create the common time trend
    delta = np.random.normal(loc=0, scale=1, size=T)

    # Create the 10 stationary components
    lambda_mat = np.zeros((T, 10))
    for k in range(10):
        lambda_mat[:, k] = np.random.normal(loc=rho, scale=sd_rho, size=T)
        
    # Create the 2 non-stationary components
    phi_mat = np.random.normal(loc=0, scale=1, size=(T, 2))
    for t in range(1, T):
        phi_mat[t, :] += phi_mat[t-1, :]    

    # Create the observed outcome
    for j in range(1, J+1):
        
        # Group indicator of the stationary component
        mu1 = np.zeros(10)
        mu1[(j-1)//2] = 1
        
        # Group indicator of the non-stationary component
        mu2 = np.zeros(2)
        mu2[j // 11] = 1
        
        # Error term
        if process == 'stationary':
            epsilon = np.random.normal(loc=0, scale=1, size=T)
        elif process == 'nonstationary':
            epsilon = np.random.normal(loc=0, scale=1, size=T)
        
        # Observed outcome
        if process == 'stationary':
            data[unit_ids[j-1]] = delta + np.dot(lambda_mat, mu1) + epsilon
        elif process == 'nonstationary':
            data[unit_ids[j-1]] = delta + np.dot(lambda_mat, mu1) + np.dot(phi_mat, mu2) + epsilon
        
    data[unit_ids[0]][T0:] += treatmenteffect*(np.arange(T0,T)-T0)
        
    y = data.iloc[:,0].values.reshape(T,1)
    X = data.iloc[:,1:]
        
    return data, y, X

### 1.4. Example Data plot <a class="anchor" id="section1_4"></a>

In [ ]:
data_stationary, y_stationary, X_stationary = dgp(J, T0, T, 'stationary', 1)
data_nonstationary, y_nonstationary, X_nonstationary = dgp(J, T0, T, 'nonstationary', 1)
placebo_units = data_stationary.columns.unique()

plt.figure(figsize=(16,5))

plt.subplot(1,2,1)
plt.plot(data_stationary['A'], color='blue')
plt.plot(data_stationary.drop(['A'], axis=1), color='grey', alpha=.5)
plt.axvline(T0, color='grey', linestyle='dashed', alpha=.75)
plt.title('Stationary DGP')

plt.subplot(1,2,2)
plt.plot(data_nonstationary['A'], color='blue')
plt.plot(data_nonstationary.drop(['A'], axis=1), color='grey', alpha=.5)
plt.axvline(T0, color='grey', linestyle='dashed', alpha=.75)
plt.title('Nonstationary DGP');

## 2. Synthetic Control Class definition and functions <a class="anchor" id="chapter2"></a>
### 2.1 Synthetic Control Class definition  <a class="anchor" id="section2_1"></a>

In [ ]:
class SyntheticControl:

    def __init__(
        self,
        outer_loop_method='Nelder-Mead',
        maxiter=maxiter,
        maxfev=maxfev,
        xatol=1e-3,
        fatol=1e-12,
        verbose_outer=True,
        verbose_inner=True,
        weight_nonnegative_restr=True,
        weight_sum_restr=True,
        count=0,
        include_intercept=False,
        demean=False,
    ):
        self.outer_loop_method = outer_loop_method
        self.maxiter = maxiter
        self.maxfev = maxfev
        self.xatol = xatol
        self.fatol = fatol
        self.verbose_outer = verbose_outer
        self.verbose_inner = verbose_inner
        self.weight_nonnegative_restr = weight_nonnegative_restr
        self.weight_sum_restr = weight_sum_restr
        self.count = count
        self.include_intercept = include_intercept
        self.demean = demean
        
    def _loss_function(self, v2, y, X):

        # Initialize matrices
        V = np.diag(np.insert(np.exp(v2), 0, 1))
        H = np.dot(X.T, np.dot(V, X))
        l = X.shape[1]

        # Create correct CVxOPT matrices for quad. optimization loop with constraints
        Q = matrix((H + H.T) / 2)
        r = matrix((-np.dot(y.T, np.dot(V, X))).T)
        G = matrix(-np.eye(l))
        h = matrix(np.zeros(l))
        A = matrix(np.ones(l)).T
        b = matrix(1.0)

        # Optimization
        if not self.weight_nonnegative_restr:
            G, h = None, None
        if not self.weight_sum_restr:
            A, b = None, None

        w = solvers.qp(Q, r, G, h, A, b, kktsolver='ldl')["x"]
        SSR = np.sum((y - np.dot(X, w)) ** 2)

        # Print progress
        self.count += 1
        if self.verbose_inner and self.count % 1000 == 0:
            print(f"Function evaluation {self.count}/{self.maxfev} \tloss = {SSR}")

        return SSR
        
    def fit(self, X, y):

        # Set starting values for outer loop
        s = np.hstack((y, X)).std(axis=1)
        s1, s2 = s[0], s[1:]
        v20 = np.log((s1/s2)**2)
        minimize_options = {'maxiter': self.maxiter, 
                            'maxfev': self.maxfev,
                            'disp': self.verbose_outer,
                            'xatol': self.xatol, 
                            'fatol': self.fatol}
        
        result = minimize(self._loss_function, v20, args=(y, X), method=self.outer_loop_method, 
              options = minimize_options)
        v2 = result.x
        
        # Restore weights from V weights
        self.V = np.diag(np.insert(np.exp(v2), 0, 1))
        H = np.dot(X.T, np.dot(self.V, X))
        l = np.shape(X)[1]
        
        # Transform to correct CVxOPT types
        Q = matrix((H + H.T)/2)
        r = matrix((-np.dot(y.T, np.dot(self.V, X))).T)
        G = matrix(-np.eye(l))
        h = matrix(np.zeros(l))
        A = matrix(np.ones(l)).T   
        b = matrix(1.0)
        
        # Optimization and resulting weights
        if not self.weight_nonnegative_restr:
            G, h  = None, None
        if not self.weight_sum_restr:
            A, b = None, None
        
        self.weights = np.array(solvers.qp(Q, r, G, h, A, b, kktsolver='ldl')['x']).T

        return self

In [ ]:
class SyntheticControlVAR:
    
    def __init__(
        self,
        weight_nonnegative_restr = True,
        weight_sum_restr = True,
        verbose_inner = False,
        verbose_outer = False,
    ):
        self.weight_nonnegative_restr = weight_nonnegative_restr
        self.weight_sum_restr = weight_sum_restr
        self.verbose_inner = verbose_inner
        self.verbose_outer = verbose_outer
       
    def fit(self, X, y):
        # Find number of donor countries
        self.n_donorcountries = np.shape(X)[1]

        # Define V matrix
        self.V = np.diag(1/np.var(X, axis=1))

        # Set up optimization for quadratic programming
        H = np.dot(X.T, np.dot(self.V, X))
        l = np.shape(X)[1]

        # Transform to correct CVxOPT types
        Q = matrix((H + H.T)/2)
        r = matrix((-np.dot(y.T, np.dot(self.V,X))).T)
        G = matrix(-np.eye(l))
        h = matrix(np.zeros(l))
        A = matrix(np.ones(l)).T
        b = matrix(1.0)

        # Optimization and resulting weights
        if not self.weight_nonnegative_restr:
            G, h  = None, None
        if not self.weight_sum_restr:
            A, b = None, None

        self.weights = np.array(solvers.qp(Q, r, G, h, A, b, kktsolver='ldl')["x"]).T[0]

        return self

In [ ]:
class SyntheticControlCV:
    
    def __init__(
        self,
        outer_loop_method = 'Nelder-Mead',
        maxiter = maxiter,
        maxfev = maxfev,
        xatol = 1e-3,
        fatol = 1e-12,
        verbose_outer = True,
        verbose_inner = True,
        weight_nonnegative_restr = True,
        weight_sum_restr = True,
        count = 0,
    ):
        self.outer_loop_method = outer_loop_method
        self.maxiter = maxiter
        self.maxfev = maxfev
        self.xatol = xatol
        self.fatol = fatol
        self.verbose_outer = verbose_outer
        self.verbose_inner = verbose_inner
        self.weight_nonnegative_restr = weight_nonnegative_restr
        self.weight_sum_restr = weight_sum_restr
        self.count = count
        
    def _loss_function(self, v2, y_train, X_train, y_validation, X_validation):
        
        # Initalize matrices
        V = np.diag(np.insert(np.exp(v2), 0, 1))
        H = np.dot(X_train.T, np.dot(V, X_train))
        l = np.shape(X_train)[1]

        # Create correct CVxOPT matrices for quad. optimization loop with constraints
        Q = matrix((H + H.T)/2)
        r = matrix((-np.dot(y_train.T, np.dot(V, X_train))).T)
        G = matrix(-np.eye(l))
        h = matrix(np.zeros(l))
        A = matrix(np.ones(l)).T
        b = matrix(1.0)

        # Optimization
        if not self.weight_nonnegative_restr:
            G, h  = None, None
        if not self.weight_sum_restr:
            A, b = None, None

        w = solvers.qp(Q, r, G, h, A, b)['x']
        SSR = np.sum((y_validation - np.dot(X_validation, w))**2)

        # Print progress
        self.count += 1
        if (self.verbose_inner) & (self.count%1000==0):
            print(f'Function evaluation {self.count}/{self.maxfev} \tloss = {SSR}')

        return SSR

    def fit(self, X, y):

        # 1. Divide the pre-intervention periods into a initial training and validation period
        # Training set contains 1-10, validation set 11-15
        y_train = y_nonstationary[:10]
        X_train = X_nonstationary.iloc[:t0, :].values
        y_validation = y_nonstationary[t0:T0]
        X_validation = X_nonstationary.iloc[t0:T0, :].values

        # Set starting values for outer loop
        s = np.hstack((y_train, X_train)).std(axis=1)
        s1, s2 = s[0], s[1:]
        v20 = np.log((s1/s2)**2)
        minimize_options = {'maxiter': self.maxiter, 
                            'maxfev': self.maxfev,
                            'disp': self.verbose_outer,
                            'xatol': self.xatol, 
                            'fatol': self.fatol}

        # Outer loop
        result = minimize(self._loss_function, v20, args=(y_train, X_train, y_validation, X_validation), 
                          method=self.outer_loop_method, options = minimize_options)
        v2 = result.x

        # Restore weights from V weights
        self.V = np.diag(np.insert(np.exp(v2), 0, 1))
        H = np.dot(X_train.T, np.dot(self.V, X_train))
        l = np.shape(X_train)[1]

        # Transform to correct CVxOPT types
        Q = matrix((H + H.T)/2)
        r = matrix((-np.dot(y_train.T, np.dot(self.V, X_train))).T)
        G = matrix(-np.eye(l))
        h = matrix(np.zeros(l))
        A = matrix(np.ones(l)).T
        b = matrix(1.0)

        # Optimization and resulting weights
        if not self.weight_nonnegative_restr:
            G, h  = None, None
        if not self.weight_sum_restr:
            A, b = None, None

        self.weights = np.array(solvers.qp(Q, r, G, h, A, b)['x']).T

        return self

### 2.2. Help functions simulation <a class="anchor" id="section2_2"></a>

In [ ]:
def gap_computer(data_X, placebo_unit, w):
    df_treated = data_X[placebo_unit]
    df_untreated = data_X.drop(placebo_unit, axis=1)
    doppelganger = df_untreated.dot(w.T)
    result = (df_treated.values - doppelganger.values.flatten())
    
    return result

def placebo_weight_and_gap_computer(data_X, method):
    gap_dict = dict()
    for placebo_unit in placebo_units:
        y = data_X[placebo_unit][:T0].values.reshape(T0, 1)
        X = data_X.iloc[:T0, data_X.columns != placebo_unit].values
        
        SCplacebo = method.fit(X, y)
        w = SCplacebo.weights

        gap_dict[placebo_unit] = gap_computer(data_X, placebo_unit, w)
        
    return gap_dict

def test_statistic(gap_dict, sign=None):
    if sign == 'positive':
        gap_dict_positive = np.clip(gap_dict, 0, None)
        gap_sum = np.sum(gap_dict_positive**2)
    elif sign == 'negative':
        gap_dict_negative = np.clip(gap_dict, None, 0)
        gap_sum = np.sum(gap_dict_negative**2)
    else:
        gap_sum = np.sum(gap_dict**2)
        
    return gap_sum/len(gap_dict)

def ratiofit_pre_post(gap_dict):
    avg_ratiofit_countrydict = dict()
    pre_treatment_fits = dict()
    post_treatment_fits = dict()
    
    for placebo_unit in placebo_units:
        pre_treatment_fit = test_statistic(gap_dict[placebo_unit][:(T0+1)])
        post_treatment_fit = test_statistic(gap_dict[placebo_unit][(T0+1):], 'positive')
        
        pre_treatment_fits[placebo_unit] = pre_treatment_fit
        post_treatment_fits[placebo_unit] = post_treatment_fit
 
        avg_ratio = post_treatment_fit/pre_treatment_fit
        avg_ratiofit_countrydict[placebo_unit] = avg_ratio
           
    return avg_ratiofit_countrydict

def SA1_placebo_weight_and_gap_computer(data_X, y_X_variable, X_X_variable, method, 
                                        y2_X_variable=None, X2_X_variable=None):
    gap_dict = dict()
    for placebo_unit in placebo_units:
        y = data_X[placebo_unit][:T0].values.reshape(T0, 1)
        X = data_X.iloc[:T0, data_X.columns != placebo_unit]
        
        new_y = np.vstack([y, y_X_variable])
        new_X = np.vstack([X.values, X_X_variable.values])
        
        if y2_X_variable is not None:
            new_y = np.vstack([new_y, y2_X_variable])
            new_X = np.vstack([new_X, X2_X_variable.values])

        SCplacebo = method.fit(new_X, new_y)
        w = SCplacebo.weights

        gap_dict[placebo_unit] = gap_computer(data_X, placebo_unit, w)

    return gap_dict

## 3. Simulation <a class="anchor" id="chapter3"></a>
## 3.1. Different Variables <a class="anchor" id="section3_1"></a>

In [ ]:
variable_results = []
for sim_no in tqdm(range(n_sims)):
    for TE_size in TE_sizes:
        
        ###### Generate data
        data_stationary, y_stationary, X_stationary = dgp(J, T0, T, 'stationary', TE_size)
        data_nonstationary, y_nonstationary, X_nonstationary = dgp(J, T0, T, 'nonstationary', TE_size)
        
        data_S_variable, y_S_variable, X_S_variable = dgp(J, T0, T0, 'stationary')
        data_N_variable, y_N_variable, X_N_variable = dgp(J, T0, T0, 'nonstationary')
        
        ###### Stationary data
        # Original method
        original_gap_dict = placebo_weight_and_gap_computer(
            data_stationary, SyntheticControl(verbose_inner=False, verbose_outer=False))
        original_te = np.round(np.sum(original_gap_dict['A'][(T0+1):])/45, 2)
        original_p_value = (20 - (pd.Series(ratiofit_pre_post(original_gap_dict).values()).rank())[0])/19
        
        # Append stationary variable
        plus_s_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_stationary, y_S_variable, X_S_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False))
        plus_s_te = np.round(np.sum(plus_s_gap_dict['A'][(T0+1):])/45, 2)
        plus_s_p_value = (20 - (pd.Series(ratiofit_pre_post(plus_s_gap_dict).values()).rank())[0])/19
        
        # Append nonstationary variable
        plus_n_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_stationary, y_N_variable, X_N_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False))
        plus_n_te = np.round(np.sum(plus_n_gap_dict['A'][(T0+1):])/45, 2)
        plus_n_p_value = (20 - (pd.Series(ratiofit_pre_post(plus_n_gap_dict).values()).rank())[0])/19
            
        # Append stationary and nonstationary variables
        plus_sn_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_stationary, y_S_variable, X_S_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False),
            y_N_variable, X_N_variable)
        plus_sn_te = np.round(np.sum(plus_sn_gap_dict['A'][(T0+1):])/45, 2)
        plus_sn_p_value = (20 - (pd.Series(ratiofit_pre_post(plus_sn_gap_dict).values()).rank())[0])/19
        
        ###### Nonstationary data
        # Original method
        N_original_gap_dict = placebo_weight_and_gap_computer(
            data_nonstationary, SyntheticControl(verbose_inner=False, verbose_outer=False))
        N_original_te = np.round(np.sum(N_original_gap_dict['A'][(T0+1):])/45, 2)
        N_original_p_value = (20 - (pd.Series(ratiofit_pre_post(N_original_gap_dict).values()).rank())[0])/19
        
        # Append stationary variable
        N_plus_s_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_nonstationary, y_S_variable, X_S_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False))
        N_plus_s_te = np.round(np.sum(N_plus_s_gap_dict['A'][(T0+1):])/45, 2)
        N_plus_s_p_value = (20 - (pd.Series(ratiofit_pre_post(N_plus_s_gap_dict).values()).rank())[0])/19
        
        # Append nonstationary variable
        N_plus_n_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_nonstationary, y_N_variable, X_N_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False))
        N_plus_n_te = np.round(np.sum(N_plus_n_gap_dict['A'][(T0+1):])/45, 2)
        N_plus_n_p_value = (20 - (pd.Series(ratiofit_pre_post(N_plus_n_gap_dict).values()).rank())[0])/19
        
        # Append stationary and nonstationary variables
        N_plus_sn_gap_dict = SA1_placebo_weight_and_gap_computer(
            data_nonstationary, y_S_variable, X_S_variable,
            SyntheticControl(verbose_inner=False, verbose_outer=False),
            y_N_variable, X_N_variable,)
        N_plus_sn_te = np.round(np.sum(N_plus_sn_gap_dict['A'][(T0+1):])/45, 2)
        N_plus_sn_p_value = (20 - (pd.Series(ratiofit_pre_post(N_plus_sn_gap_dict).values()).rank())[0])/19
        
        # Store TE and p-values
        variable_results.append({'Simulation': sim_no+1, 'TE_size': TE_size,
                                 'S TE': original_te, 'S p-val':original_p_value,
                                 'S+S TE': plus_s_te, 'S+S p-val':plus_s_p_value,
                                 'S+N TE': plus_n_te, 'S+N p-val':plus_n_p_value,
                                 'S+S+N TE': plus_sn_te, 'S+S+N p-val':plus_sn_p_value,
                                 'N TE': N_original_te, 'N p-val':N_original_p_value,
                                 'N+S TE': N_plus_s_te, 'N+S p-val':N_plus_s_p_value,
                                 'N+N TE': N_plus_n_te, 'N+N p-val':N_plus_n_p_value,
                                 'N+S+N TE': N_plus_sn_te, 'N+S+N p-val':N_plus_sn_p_value
                                })

In [ ]:
df_variable_results = pd.DataFrame(variable_results)
np.round(df_variable_results.groupby(['TE_size']).mean().drop('Simulation', axis=1), 3).T

In [ ]:
df_variable_results_to_latex = np.round(
    df_variable_results.groupby(['TE_size']).mean().drop('Simulation', axis=1), 3).T
print(df_variable_results_to_latex.style.to_latex())

## 3.2. Different V Matrices <a class="anchor" id="section3_2"></a>

In [ ]:
V_results = []
for sim_no in tqdm(range(n_sims)):
    for TE_size in TE_sizes:
        
        ###### Generate data
        data_stationary, y_stationary, X_stationary = dgp(J, T0, T, 'stationary', TE_size)
        data_nonstationary, y_nonstationary, X_nonstationary = dgp(J, T0, T, 'nonstationary', TE_size)

        ###### Stationary data
        # Original method
        try:
            original_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControl(verbose_inner=False, verbose_outer=False))
            original_te = np.round(np.sum(original_gap_dict['A'][(T0+1):])/45, 2)
            original_p_value = (20 - (pd.Series(ratiofit_pre_post(original_gap_dict).values()).rank())[0])/19
        except:
            original_te, original_p_value = np.nan, np.nan
        
        # Inverted Variance method
        try:
            invv_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControlVAR(verbose_inner=False, verbose_outer=False))
            invv_te = np.round(np.sum(invv_gap_dict['A'][(T0+1):])/45, 2)
            invv_p_value = (20 - (pd.Series(ratiofit_pre_post(invv_gap_dict).values()).rank())[0])/19
        except:
            invv_te, invv_p_value = np.nan, np.nan
        
        # Cross-validation method
        try:
            cv_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControlCV(verbose_inner=False, verbose_outer=False))
            cv_te = np.round(np.sum(cv_gap_dict['A'][(T0+1):])/45, 2)
            cv_p_value = (20 - (pd.Series(ratiofit_pre_post(cv_gap_dict).values()).rank())[0])/19
        except:
            cv_te, cv_p_value = np.nan, np.nan
        
        ###### Nonstationary data
        # Original method
        try:
            N_original_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControl(verbose_inner=False, verbose_outer=False))
            N_original_te = np.round(np.sum(N_original_gap_dict['A'][(T0+1):])/45, 2)
            N_original_p_value = (20 - (pd.Series(ratiofit_pre_post(N_original_gap_dict).values()).rank())[0])/19
        except:
            N_original_te, N_original_p_value = np.nan, np.nan
        
        # Inverted Variance method
        try:
            N_invv_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControlVAR(verbose_inner=False, verbose_outer=False))
            N_invv_te = np.round(np.sum(N_invv_gap_dict['A'][(T0+1):])/45, 2)
            N_invv_p_value = (20 - (pd.Series(ratiofit_pre_post(N_invv_gap_dict).values()).rank())[0])/19
        except:
            N_invv_te, N_invv_p_value = np.nan, np.nan
        
        # Cross-validation method
        try:
            N_cv_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControlCV(verbose_inner=False, verbose_outer=False))
            N_cv_te = np.round(np.sum(N_cv_gap_dict['A'][(T0+1):])/45, 2)
            N_cv_p_value = (20 - (pd.Series(ratiofit_pre_post(N_cv_gap_dict).values()).rank())[0])/19
        except:
            N_cv_te, N_cv_p_value = np.nan, np.nan
        
        # Store TE and p-values
        V_results.append({'Simulation': sim_no+1, 'TE_size': TE_size,
                          'O TE': original_te, 'O p-val':original_p_value,
                          'IV TE': invv_te, 'IV p-val':invv_p_value,
                          'CV TE': cv_te, 'CV p-val':cv_p_value,
                          'N. O TE': N_original_te, 'N. O p-val':N_original_p_value,
                        'N. IV TE': N_invv_te, 'N. IV p-val':N_invv_p_value,
                        'N. CV TE': N_cv_te, 'N. CV p-val':N_cv_p_value,
                       })

In [ ]:
df_V_results = pd.DataFrame(V_results)
np.round(df_V_results.groupby(['TE_size']).mean().drop('Simulation', axis=1), 3).T

In [ ]:
df_V_to_latex = np.round(df_V_results.groupby(['TE_size']).mean().drop('Simulation', axis=1),3).T
print(df_V_to_latex.style.to_latex())

## 3.3. Different Weight Constraints <a class="anchor" id="section3_3"></a>

In [ ]:
n_sims=10
W_results = []
for sim_no in tqdm(range(n_sims)):
    for TE_size in TE_sizes:
        
        ###### Generate data
        data_stationary, y_stationary, X_stationary = dgp(J, T0, T, 'stationary', TE_size)
        data_nonstationary, y_nonstationary, X_nonstationary = dgp(J, T0, T, 'nonstationary', TE_size)

        ###### Stationary data
        # Original method
        try:
            original_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=True, weight_sum_restr=True))
            original_te = np.round(np.sum(original_gap_dict['A'][(T0+1):])/45, 2)
            original_p_value = (20 - (pd.Series(ratiofit_pre_post(original_gap_dict).values()).rank())[0])/19
        except:
            original_te, original_p_value = np.nan, np.nan
            
        # No nonnegativity constraint
        try:
            nonn_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=False, weight_sum_restr=True))
            nonn_te = np.round(np.sum(nonn_gap_dict['A'][(T0+1):])/45, 2)
            nonn_p_value = (20 - (pd.Series(ratiofit_pre_post(nonn_gap_dict).values()).rank())[0])/19
        except:
            nonn_te, nonn_p_value = np.nan, np.nan
        
        # No sum constraint
        try:
            nosum_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=True, weight_sum_restr=False))
            nosum_te = np.round(np.sum(nosum_gap_dict['A'][(T0+1):])/45, 2)
            nosum_p_value = (20 - (pd.Series(ratiofit_pre_post(nosum_gap_dict).values()).rank())[0])/19
        except:
            nosum_te, nosum_p_value = np.nan, np.nan

        # Unconstrained
        try:
            unconstr_gap_dict = placebo_weight_and_gap_computer(
                data_stationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=False, weight_sum_restr=False))
            unconstr_te = np.round(np.sum(unconstr_gap_dict['A'][(T0+1):])/45, 2)
            unconstr_p_value = (20 - (pd.Series(ratiofit_pre_post(unconstr_gap_dict).values()).rank())[0])/19
        except:
            unconstr_te, unconstr_p_value = np.nan, np.nan

        ###### Nonstationary data
        # Original method
        try:
            N_original_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=True, weight_sum_restr=True))
            N_original_te = np.round(np.sum(N_original_gap_dict['A'][(T0+1):])/45, 2)
            N_original_p_value = (20 - (pd.Series(ratiofit_pre_post(N_original_gap_dict).values()).rank())[0])/19
        except:
            N_original_te, N_original_p_value = np.nan, np.nan
        
        # No nonnegativity constraint
        try:
            N_nonn_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=False, weight_sum_restr=True))
            N_nonn_te = np.round(np.sum(N_nonn_gap_dict['A'][(T0+1):])/45, 2)
            N_nonn_p_value = (20 - (pd.Series(ratiofit_pre_post(N_nonn_gap_dict).values()).rank())[0])/19
        except:
            N_nonn_te, N_nonn_p_value = np.nan, np.nan
        
        # No sum constraint
        try:
            N_nosum_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=True, weight_sum_restr=False))
            N_nosum_te = np.round(np.sum(N_nosum_gap_dict['A'][(T0+1):])/45, 2)
            N_nosum_p_value = (20 - (pd.Series(ratiofit_pre_post(N_nosum_gap_dict).values()).rank())[0])/19
        except:
            N_nosum_te, N_nosum_p_value = np.nan, np.nan

        # Unconstrained
        try:
            N_unconstr_gap_dict = placebo_weight_and_gap_computer(
                data_nonstationary, SyntheticControl(
                    verbose_inner=False, verbose_outer=False, weight_nonnegative_restr=False, weight_sum_restr=False))
            N_unconstr_te = np.round(np.sum(N_unconstr_gap_dict['A'][(T0+1):])/45, 2)
            N_unconstr_p_value = (20 - (pd.Series(ratiofit_pre_post(N_unconstr_gap_dict).values()).rank())[0])/19
        except:
            N_unconstr_te, N_unconstr_p_value = np.nan, np.nan
        
        # Store TE and p-values
        W_results.append({'Simulation': sim_no+1, 'TE_size': TE_size,
                          'O TE': original_te, 'O p-val':original_p_value,
                          'NN TE': nonn_te, 'NN p-val':nonn_p_value,
                          'S TE': nosum_te, 'S p-val':nosum_p_value,
                          'U TE': unconstr_te, 'U p-val':unconstr_p_value,
                          'N O TE': N_original_te, 'N O p-val':N_original_p_value,
                          'N NN TE': N_nonn_te, 'N NN p-val':N_nonn_p_value,
                          'N S TE': N_nosum_te, 'N S p-val':N_nosum_p_value,
                          'N U TE': N_unconstr_te, 'N U p-val':N_unconstr_p_value
                       })

In [ ]:
df_W_results = pd.DataFrame(W_results)
np.round(df_W_results.groupby(['TE_size']).mean().drop('Simulation', axis=1), 3).T

In [ ]:
df_W_to_latex = np.round(df_W_results.groupby(['TE_size']).mean().drop('Simulation', axis=1),3).T
print(df_W_to_latex.style.to_latex())